#### Define the Tree Structure

In [ ]:
class Node:
    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None


#### Gini Impurity Calculation

In [ ]:
def gini_impurity(y):
    m = len(y)
    return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in np.unique(y))


#### Split the Dataset

In [ ]:
def split_dataset(X, y, index, threshold):
    left_mask = X[:, index] < threshold
    right_mask = X[:, index] >= threshold
    return X[left_mask], X[right_mask], y[left_mask], y[right_mask]


#### Find the Best Split

In [ ]:
def best_split(X, y):
    m, n = X.shape
    if m <= 1:
        return None, None

    num_parent = [np.sum(y == c) for c in np.unique(y)]
    best_gini = 1.0 - sum((num / m) ** 2 for num in num_parent)
    best_index, best_threshold = None, None

    for index in range(n):
        thresholds, classes = zip(*sorted(zip(X[:, index], y)))
        num_left = [0] * len(np.unique(y))
        num_right = num_parent.copy()

        for i in range(1, m):
            c = classes[i - 1]
            num_left[c] += 1
            num_right[c] -= 1
            gini_left = 1.0 - sum((num_left[x] / i) ** 2 for x in np.unique(y))
            gini_right = 1.0 - sum((num_right[x] / (m - i)) ** 2 for x in np.unique(y))

            gini = (i * gini_left + (m - i) * gini_right) / m

            if thresholds[i] == thresholds[i - 1]:
                continue

            if gini < best_gini:
                best_gini = gini
                best_index = index
                best_threshold = (thresholds[i] + thresholds[i - 1]) / 2

    return best_index, best_threshold


#### Build the Tree

In [ ]:
def build_tree(X, y, depth=0, max_depth=10):
    num_samples_per_class = [np.sum(y == i) for i in np.unique(y)]
    predicted_class = np.argmax(num_samples_per_class)
    node = Node(
        gini=gini_impurity(y),
        num_samples=len(y),
        num_samples_per_class=num_samples_per_class,
        predicted_class=predicted_class,
    )

    if depth < max_depth:
        index, threshold = best_split(X, y)
        if index is not None:
            X_left, X_right, y_left, y_right = split_dataset(X, y, index, threshold)
            node.feature_index = index
            node.threshold = threshold
            node.left = build_tree(X_left, y_left, depth + 1, max_depth)
            node.right = build_tree(X_right, y_right, depth + 1, max_depth)
    return node


#### Prediction

In [ ]:
def predict_sample(node, sample):
    if node.left is None and node.right is None:
        return node.predicted_class
    if sample[node.feature_index] < node.threshold:
        return predict_sample(node.left, sample)
    else:
        return predict_sample(node.right, sample)

def predict(tree, X):
    return [predict_sample(tree, sample) for sample in X]
